# A Simple Deployment

Let's take an openly available model from Github and wrap it in a simple UI!

[DM-Count](https://github.com/cvlab-stonybrook/DM-Count) is a model used for crowd counting.

Lets download the code and load the model!

In [ ]:
!git clone https://github.com/cvlab-stonybrook/DM-Count

In [ ]:
import torch
import numpy as np
import gdown
import sys
import os

sys.path.insert(0, os.path.join(os.getcwd(), 'DM-Count/'))

import models

model_path = "model.pth"
url = "https://drive.google.com/uc?id=1nnIHPaV9RGqK8JHL645zmRvkNrahD9ru"
gdown.download(url, model_path, quiet=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load model
model = models.vgg19() # DM-Count is VGG19 based
model.load_state_dict(torch.load(model_path, device))
model.eval()

# send model to compute device
model = model.to(device)

## Inference Function

Wrapping a single function that takes the input and returns the desired output is good programming practice.

Below the `detect_crowd` function takes in an image and uses the "DMCount" model instantiated earler for detecting people in the image.

Notice in this section we have a "pre processing", "inference", and "post-processing" section. These are the typical stages of "serving" a model's funcationality.

In [ ]:
import cv2

target_input_width = 1280
target_input_height = 800

def detect_crowd(original_image):
    '''
    Function for counting crowds in a single image
    '''
        
    resized_image = cv2.resize(original_image, (target_input_width, target_input_height))
    
    # preprocessing    
    image_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    image_tensor = np.float32(image_rgb/255)
    image_tensor = np.moveaxis(image_tensor, -1, 0)  # HWC to CHW
    
    image_tensor = image_tensor[np.newaxis, :] # add batch dimension

    image_tensor = torch.tensor(image_tensor)
    image_tensor = image_tensor.to(device) # move image data to compute device
    
    # inference
    with torch.no_grad():
        output, _ = model(image_tensor)
        output = output.cpu().numpy()
            
    # post processing
    crowd_count = int(np.sum(output).item())
    
    heatmap = output[0, 0]
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-5)
    heatmap = (heatmap * 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (target_input_width, target_input_height))
        
    overlayed = cv2.addWeighted(resized_image, 0.7, heatmap, 0.5, 0)

    return overlayed, crowd_count
    

## Simple UI

[Gradio](https://gradio.app/) is a fantastic little tool for building sample apps that show off ML functionality. Below is a simple interface that requires an image, and annotates a single image.

In [ ]:
import gradio as gr

gr.close_all() # cleanup any stray samples

iface = gr.Interface(fn=detect_crowd,
                     inputs=[
                         gr.Image(label="Image of Crowd"),
                     ],
                     outputs=[
                         gr.Image(label="Predicted Density Map"),
                         gr.Label(label="Predicted Count"),
                     ],
                     examples=[
                         ["sample_images/busy-road.jpg"],
                         ["sample_images/concert-crowd.jpg"],
                         ["sample_images/group-photo.jpg"],
                         ["sample_images/mountains.jpg"],
                     ],
                     title="Crowd Detection App",
                     description="A simple app tp find and count faces in a crowd",
                     )
iface.launch(server_name="0.0.0.0", server_port=7860)

In [ ]:
# cleanup

import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)